# Backtesting Fibonacci Retracement Post Wave 1 Peak

In [24]:
import requests
import sys
import json
import base64
import time
import logging
from solders.keypair import Keypair
from solders.transaction import VersionedTransaction 
from solana.rpc.api import Client
from solana.rpc.types import TxOpts
import dontshare as d 
from pprint import pprint
from functools import lru_cache
from datetime import datetime, timedelta
import pytz
import json
import pandas as pd
from pprint import pprint 
from IPython.display import display, HTML
import time

# Wallet and API Key

In [25]:
API_Key = d.birdeye
wallet = d.sol_wallet
chain = "solana"

# Filter Tokens by Market Cap Range

In [26]:
def get_token_list(sort_by, sort_type, min_liquidity, min_volume_24h, min_market_cap, max_market_cap, total_tokens, chain, API_Key):
    """
    Fetch and filter a list of tokens based on specified criteria.

    Args:
        sort_by (str): Criterion to sort by (e.g., 'mc', 'v24hUSD', 'v24hChangePercent').
        sort_type (str): Sort order ('asc' or 'desc').
        min_liquidity (float): Minimum liquidity in USD.
        min_volume_24h (float): Minimum 24-hour trading volume in USD.
        min_market_cap (float): Minimum market cap in USD.
        max_market_cap (float): Maximum market cap in USD.
        total_tokens (int): Number of tokens to retrieve.
        chain (str): Blockchain to query.
        API_Key (str): Your Birdeye API key.

    Returns:
        pd.DataFrame: A DataFrame containing the filtered token list.
        list: A list of error messages encountered during fetching.
    """
    limit = 50
    all_tokens = []
    errors = []

    headers = {
        "accept": "application/json",
        "x-chain": chain,
        "X-API-KEY": API_Key
    }

    for offset in range(0, total_tokens * 3, limit):  # Fetch more tokens to account for filtering
        url = f"https://public-api.birdeye.so/defi/tokenlist?sort_by={sort_by}&sort_type={sort_type}&offset={offset}&limit={limit}&min_liquidity={min_liquidity}"

        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                data = response.json()
                if 'data' in data and 'tokens' in data['data']:
                    new_tokens = [
                        {
                            'name': token.get('name'),
                            'symbol': token.get('symbol'),
                            'mc': token.get('mc'),
                            'v24hUSD': token.get('v24hUSD'),
                            'v24hChangePercent': token.get('v24hChangePercent'),
                            'liquidity': token.get('liquidity'),
                            'address': token.get('address')
                        }
                        for token in data['data']['tokens']
                        if token.get('v24hUSD', 0) >= min_volume_24h
                        and min_market_cap <= token.get('mc', 0) <= max_market_cap
                    ]
                    all_tokens.extend(new_tokens)
                    logging.info(f"Fetched {len(new_tokens)} tokens at offset {offset}. Total tokens collected: {len(all_tokens)}")
                else:
                    error_msg = f"Unexpected response structure at offset {offset}"
                    logging.warning(error_msg)
                    errors.append(error_msg)
                    break
            elif response.status_code == 429:
                # Rate limit exceeded, wait and retry
                retry_after = int(response.headers.get("Retry-After", 60))
                error_msg = f"Rate limit exceeded. Waiting for {retry_after} seconds."
                logging.warning(error_msg)
                errors.append(error_msg)
                time.sleep(retry_after)
                continue
            elif response.status_code == 400:
                # Bad Request - log the response content for debugging
                try:
                    error_content = response.json()
                except json.JSONDecodeError:
                    error_content = response.text
                error_msg = f"Bad Request (400) at offset {offset}: {error_content}"
                logging.error(error_msg)
                errors.append(error_msg)
                break  # Exit the loop on error
            else:
                error_msg = f"Request failed with status code: {response.status_code} at offset {offset}"
                logging.error(error_msg)
                errors.append(error_msg)
                break  # Exit the loop on error
        except Exception as e:
            error_msg = f"An exception occurred: {e}"
            logging.error(error_msg)
            errors.append(error_msg)
            break  # Exit on exception

        if len(all_tokens) >= total_tokens:
            break

        # Respect rate limits
        time.sleep(1)

    all_tokens = all_tokens[:total_tokens]

    if all_tokens:
        df = pd.DataFrame(all_tokens)

        # Select and rename columns
        columns_to_display = {
            'name': 'Name',
            'symbol': 'Symbol',
            'mc': 'Market Cap',
            'v24hUSD': 'Volume 24h',
            'v24hChangePercent': '24h Change (%)',
            'liquidity': 'Liquidity',
            'address': 'Address'
        }
        
        # Only include columns that exist in the DataFrame
        available_columns = [col for col in ['name', 'symbol', 'mc', 'v24hUSD', 'v24hChangePercent', 'liquidity', 'address'] if col in df.columns]
        df = df[available_columns].rename(columns={col: columns_to_display[col] for col in available_columns})

        return df, errors
    else:
        return pd.DataFrame(), errors

In [27]:
# Example usage:
sort_by = "mc"  # mc, v24hUSD, v24hChangePercent 
sort_type = "desc"  # asc, desc
min_liquidity = 100000  # Minimum liquidity in USD (100k)
min_volume_24h = 1000000  # Minimum 24-hour trading volume in USD
min_market_cap = 1000000  # Minimum market cap in USD (1 million)
max_market_cap = 900000000  # Maximum market cap in USD (900 Million)
total_tokens = 200  # Change this to the number of tokens you want to retrieve
chain = "solana"  # Choose between: solana,ethereum,arbitrum,avalanche,bsc,optimism,polygon,base,zksync

filtered_tokens = get_token_list(sort_by, sort_type, min_liquidity, min_volume_24h, min_market_cap, max_market_cap, total_tokens, chain, API_Key)

print(filtered_tokens[0])

(                            Name     Symbol    Market Cap    Volume 24h  \
0     Marinade staked SOL (mSOL)       mSOL  8.170247e+08  9.749055e+06   
1            cat in a dogs world        MEW  7.733612e+08  5.205914e+06   
2               Jupiter Perps LP        JLP  7.509179e+08  1.865687e+07   
3                   BOOK OF MEME       BOME  6.415667e+08  5.881563e+06   
4                           Kima       KIMA  5.740995e+08  9.565674e+07   
5             Jupiter Staked SOL     JupSOL  5.654557e+08  8.772435e+06   
6               Goatseus Maximus       GOAT  4.706266e+08  1.342389e+08   
7                       GIGACHAD       GIGA  4.527847e+08  9.169922e+06   
8                  ANGER TADPOLE       GELE  4.195313e+08  1.201345e+06   
9                          GRA$$      GRA$$  3.668662e+08  3.343884e+08   
10                      deBridge        DBR  3.447399e+08  3.015576e+06   
11                      deBr1dge        DBR  2.830875e+08  1.422299e+08   
12  BlazeStake Staked SO

# Get Token List

#### Established Tokens

In [32]:
established_tokens_filtered = filtered_tokens[0]['Address'].tolist() 
print(established_tokens_filtered[:5])

['mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So', 'MEW1gQWJ3nEXg2qgERiKu7FAFj79PHvQVREQUzScPP5', '27G8MtK7VtTcCHkpASjSDdkWWYfoqT6ggEuKidVJidD4', 'ukHH6c7mMyiWCf1b9pnWe25TSpkDDt3H5pQZgZ74J82', 'BsQCC4D2AZhC9RctuugBKLCWaNycwmZTzwpUjgGHXWbw']


#### New Token Listings

In [29]:
def get_new_listings(days_back, hours_back, minutes_back, API_Key, liquidity_filter):
    """
    Fetch new token listings data.

    Args:
    days_back (int): Number of days to look back (1, 2, or 3).
    hours_back (int): Number of hours to look back within the selected day.
    minutes_back (int): Number of minutes to look back within the selected hour.
    API_Key (str): Your Birdeye API key.
    liquidity_filter (float): Minimum liquidity for filtered results.

    Returns:
    tuple: A tuple containing two DataFrames (full_df, filtered_df) with new listings data.
    """
    def iso8601_to_timestamp(iso_string):
        return int(datetime.fromisoformat(iso_string.replace('Z', '+00:00')).timestamp())

    end_time = datetime.now(pytz.UTC)
    
    if days_back == 1:
        start_time = end_time - timedelta(hours=hours_back, minutes=minutes_back)
    elif days_back == 2:
        start_time = end_time - timedelta(days=1, hours=hours_back, minutes=minutes_back)
    elif days_back == 3:
        start_time = end_time - timedelta(days=2, hours=hours_back, minutes=minutes_back)
    else:
        raise ValueError("days_back must be 1, 2, or 3")
    
    all_tokens = []
    current_time = int(end_time.timestamp())
    
    while current_time > int(start_time.timestamp()):
        url = f"https://public-api.birdeye.so/defi/v2/tokens/new_listing?time_to={current_time}&limit=10"

        headers = {
            "accept": "application/json",
            "X-API-KEY": API_Key
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            if 'data' in data and 'items' in data['data']:
                new_tokens = data['data']['items']
                if not new_tokens:
                    break
                all_tokens.extend(new_tokens)
                current_time = min(iso8601_to_timestamp(token['liquidityAddedAt']) for token in new_tokens)
            else:
                print(f"Unexpected response structure")
                print("Response:", response.text)
                break
        else:
            print(f"Request failed with status code: {response.status_code}")
            print("Response:", response.text)
            break

        # Respect rate limits
        time.sleep(1)

    # Remove duplicates based on 'address'
    unique_tokens = list({token['address']: token for token in all_tokens}.values())
    
    if unique_tokens:
        # Create full DataFrame
        df_full = pd.DataFrame(unique_tokens)
        
        # Convert liquidityAddedAt to datetime
        df_full['liquidityAddedAt'] = pd.to_datetime(df_full['liquidityAddedAt'])
        
        # Sort by liquidityAddedAt in descending order
        df_full = df_full.sort_values('liquidityAddedAt', ascending=False)
        
        # Apply filters for the second table
        df_filtered = df_full[
            (df_full['name'].notna() & (df_full['name'] != 'None')) & 
            (df_full['logoURI'].notna() & (df_full['logoURI'] != 'None')) & 
            (df_full['liquidity'].astype(float) >= liquidity_filter)
        ]
        
        return df_full, df_filtered
    else:
        return pd.DataFrame(), pd.DataFrame()

# Example usage:
days_back = 1  # Change this value to look back fewer days (1 for 1 day, 2 for 2 days, 3 for 3 days max)
hours_back = 0  # Change this value to look back fewer hours within the selected day
                # Note: when hours_back is 1, it looks back for all new tokens within 1 hour.
                # when hours_back is 0, it looks back for all new tokens within the selected minutes.
minutes_back = 5  # Change this value to look back fewer minutes within the selected hour
liquidity_filter = 10000
new_tokens_filtered = get_new_listings(days_back, hours_back, minutes_back, API_Key, liquidity_filter)
#print(df_filtered)
print(new_tokens_filtered[0].columns.tolist())


['address', 'symbol', 'name', 'decimals', 'liquidityAddedAt', 'logoURI', 'liquidity']


# Get Token OHLCV and Save

In [30]:
def get_ohlcv_data_multi(tokens, API_Key, timeframes=None):
    """
    Fetch OHLCV data for multiple tokens and specified timeframes.

    Args:
    tokens (list): List of token addresses to fetch data for.
    API_Key (str): Your Birdeye API key.
    timeframes (list): List of timeframes to fetch data for. Default is ['15m'].

    Returns:
    dict: A nested dictionary with tokens and timeframes as keys and DataFrames as values.
    """
    if timeframes is None:
        timeframes = ['15m']

    timeframes_data = {
        '1m': 1 * 24 * 60,    # 1 day of 1-minute data
        '3m': 3 * 24 * 20,    # 3 days of 3-minute data
        '5m': 7 * 24 * 12,    # 7 days of 5-minute data
        '15m': 25 * 24 * 4,   # 25 days of 15-minute data
        '30m': 50 * 24 * 2,   # 50 days of 30-minute data
        '1H': 100 * 24,       # 100 days of 1-hour data
        '2H': 200 * 12,       # 200 days of 2-hour data
        '4H': 400 * 6,        # 400 days of 4-hour data
        '6H': 600 * 4,        # 600 days of 6-hour data
        '8H': 800 * 3,        # 800 days of 8-hour data
        '12H': 1200 * 2,      # 1200 days of 12-hour data
        '1D': 5 * 365         # 5 years of daily data
    }

    results = {}

    for token in tokens:
        results[token] = {}
        for timeframe in timeframes:
            if timeframe not in timeframes_data:
                print(f"Invalid timeframe: {timeframe}. Skipping.")
                continue

            # Calculate start and end times
            end_time = datetime.now(pytz.UTC)
            start_time = end_time - timedelta(hours=timeframes_data[timeframe])

            # Convert to Unix timestamps
            time_from = int(start_time.timestamp())
            time_to = int(end_time.timestamp())

            url = f"https://public-api.birdeye.so/defi/ohlcv?address={token}&type={timeframe}&time_from={time_from}&time_to={time_to}"

            headers = {
                "accept": "application/json",
                "X-API-KEY": API_Key
            }

            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                try:
                    data = json.loads(response.text)
                    
                    if 'data' in data and 'items' in data['data']:
                        items = data['data']['items']
                        
                        if items:
                            df = pd.DataFrame(items)
                            df['datetime'] = pd.to_datetime(df['unixTime'], unit='s')
                            df.set_index('datetime', inplace=True)
                            columns_order = [col for col in df.columns if col != 'unixTime']
                            df = df[columns_order]

                            results[token][timeframe] = df
                            
                            # Print the head of each token-timeframe table
                            print(f"\nHead of {token} - {timeframe} table:")
                            print(df.head())
                        else:
                            results[token][timeframe] = pd.DataFrame()
                            print(f"\nNo data items found for token {token}, timeframe {timeframe}.")
                    else:
                        results[token][timeframe] = pd.DataFrame()
                        print(f"\nUnexpected response structure for token {token}, timeframe {timeframe}.")
                except json.JSONDecodeError:
                    results[token][timeframe] = pd.DataFrame()
                    print(f"\nFailed to parse JSON response for token {token}, timeframe {timeframe}.")
            else:
                results[token][timeframe] = pd.DataFrame()
                print(f"\nRequest failed with status code: {response.status_code} for token {token}, timeframe {timeframe}.")

    return results


In [31]:
# Example usage:
tokens = filtered_token_list
result = get_ohlcv_data_multi(tokens, API_Key, timeframes=['1m']) # ['1m', '3m', '5m', '15m', '30m', '1H', '2H', '4H', '6H', '8H', '12H', '1D']


Head of mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So - 1m table:
                                                         address           c  \
datetime                                                                       
2024-08-20 17:13:00  mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So  173.034117   
2024-08-20 17:14:00  mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So  173.008842   
2024-08-20 17:15:00  mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So  173.014879   
2024-08-20 17:16:00  mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So  173.091518   
2024-08-20 17:17:00  mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So  173.122128   

                              h           l           o type          v  
datetime                                                                 
2024-08-20 17:13:00  173.120417  173.034117  173.077397   1m   6.944023  
2024-08-20 17:14:00  173.034117  173.008842  173.034117   1m  12.035376  
2024-08-20 17:15:00  173.078423  172.988920  173.008842   1m  91.484238  
2024

# Implement Fibonacci Retracement Optimization